# ĐỒ ÁN MÔN HỌC MÁY HỌC
## Phân tích và dự đoán chất lượng rượu vang (Wine Quality – UCI)

**Dataset:** Wine Quality (UCI Machine Learning Repository)  
**Link:** https://archive.ics.uci.edu/dataset/186/wine+quality

---

## Checklist bắt buộc
### EDA
- info/describe, missing
- 11 histogram features
- correlation heatmap
- quality distribution + label distribution
- nhận xét đặc trưng ảnh hưởng

### Preprocessing
- missing explicit (SimpleImputer)
- outliers explicit (IQR clipping) leak-free
- scaling Standard/MinMax
- split test_size 0.1/0.2/0.3 loop
- label: 0 (≤5), 1 (≥6)

### Models
- Logistic Regression, Decision Tree, LightGBM, XGBoost

### Evaluation
- Accuracy, Precision, Recall, F1
- Confusion matrix
- ROC curve + AUC
- bảng so sánh mô hình

### Feature importance
- LGBM + XGBoost

## Optional 
- Train  Red vs White
- Regression (RMSE/MAE/R2)
- Cross-validation mean±std
- Tuning (RandomizedSearchCV)


## 1) Import & cấu hình


In [ ]:
# (Tuỳ chọn nếu thiếu trên Colab)
# !pip -q install lightgbm xgboost

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc,
    mean_squared_error, mean_absolute_error, r2_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor

RANDOM_STATE = 42
TEST_SIZES = (0.1, 0.2, 0.3)       
LABEL_THRESHOLD = 6                # label=1 nếu quality>=6
SCALER_MODE = "standard"           # "standard" hoặc "minmax"
CV_FOLDS = 5                       # optional: 5-fold 
IQR_FACTOR = 1.5                   # outlier clipping


## 2) (Tuỳ chọn) Tải dataset tự động nếu chưa có file CSV


In [ ]:
import os, urllib.request

base = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/"
files = {
    "winequality-red.csv": base + "winequality-red.csv",
    "winequality-white.csv": base + "winequality-white.csv",
}

for fn, url in files.items():
    if not os.path.exists(fn):
        print("Downloading:", fn)
        urllib.request.urlretrieve(url, fn)

print("Files present:", [fn for fn in files if os.path.exists(fn)])


## 3) Data Ingestion: đọc đỏ + trắng, concat, thêm wine_type


In [ ]:
RED_PATH = "winequality-red.csv"
WHITE_PATH = "winequality-white.csv"
sep = ";"

df_red = pd.read_csv(RED_PATH, sep=sep)
df_white = pd.read_csv(WHITE_PATH, sep=sep)

df_red["wine_type"] = "red"
df_white["wine_type"] = "white"

df = pd.concat([df_red, df_white], ignore_index=True)
print("Shape:", df.shape)
df.head()


## 4) EDA: info/describe + missing values


In [ ]:
display(df.info())
display(df.describe())

missing = df.isna().sum().sort_values(ascending=False)
display(missing[missing > 0])
print("Total missing:", int(missing.sum()))


## 5) EDA: phân bố quality + nhãn nhị phân (0:<=5, 1:>=6)


In [ ]:
plt.figure()
df["quality"].hist(bins=20)
plt.title("Distribution of quality")
plt.xlabel("quality"); plt.ylabel("count")
plt.tight_layout()
plt.show()

df["label"] = (df["quality"] >= LABEL_THRESHOLD).astype(int)
display(df["label"].value_counts(normalize=True).rename("proportion"))

plt.figure()
df["label"].value_counts().sort_index().plot(kind="bar")
plt.title("Binary label distribution (0:<=5, 1:>=6)")
plt.xlabel("label"); plt.ylabel("count")
plt.tight_layout()
plt.show()


## 6) EDA: Phân bố từng đặc trưng (11 histogram)


In [ ]:
feature_cols = [c for c in df.columns if c not in ["quality","label","wine_type"]]

cols = 4
rows = int(np.ceil(len(feature_cols)/cols))
plt.figure(figsize=(16, 10))

for i, col in enumerate(feature_cols, 1):
    plt.subplot(rows, cols, i)
    plt.hist(df[col].dropna(), bins=30)
    plt.title(col, fontsize=10)
    plt.xlabel(col, fontsize=8)
    plt.ylabel("count", fontsize=8)

plt.tight_layout()
plt.show()


## 7) EDA: Correlation heatmap


In [ ]:
num_cols = df.select_dtypes(include=[np.number]).columns
corr = df[num_cols].corr()

plt.figure(figsize=(10, 8))
plt.imshow(corr, aspect="auto")
plt.title("Correlation heatmap (imshow)")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90, fontsize=8)
plt.yticks(range(len(corr.index)), corr.index, fontsize=8)
plt.colorbar()
plt.tight_layout()
plt.show()


## 8) EDA: Outliers minh hoạ 


In [ ]:
plt.figure(figsize=(18, 6))
df[feature_cols].boxplot(rot=90)
plt.title("Boxplot BEFORE outlier handling")
plt.tight_layout()
plt.show()


## 9) Preprocessing leak-free: Imputer + IQR clipping + Scaler + OneHot


In [ ]:
class IQRClipper(BaseEstimator, TransformerMixin):
    """Clip outliers theo IQR (không drop mẫu), leak-free khi nằm trong Pipeline."""
    def __init__(self, factor=1.5):
        self.factor = factor

    def fit(self, X, y=None):
        X = np.asarray(X, dtype=float)
        self.q1_ = np.nanpercentile(X, 25, axis=0)
        self.q3_ = np.nanpercentile(X, 75, axis=0)
        self.iqr_ = self.q3_ - self.q1_
        self.lower_ = self.q1_ - self.factor * self.iqr_
        self.upper_ = self.q3_ + self.factor * self.iqr_
        return self

    def transform(self, X):
        X = np.asarray(X, dtype=float)
        return np.clip(X, self.lower_, self.upper_)

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return None
        return np.array(input_features, dtype=object)


def make_preprocess(X: pd.DataFrame):
    numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

    scaler = StandardScaler() if SCALER_MODE == "standard" else MinMaxScaler()

    num_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("iqr", IQRClipper(factor=IQR_FACTOR)),
        ("scaler", scaler),
    ])

    cat_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore")),
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", num_pipe, numeric_features),
            ("cat", cat_pipe, categorical_features),
        ],
        remainder="drop"
    )
    return preprocess


## 10) 4 mô hình ML (LogReg, DecisionTree, LGBM, XGBoost)


In [ ]:
def build_models(preprocess):
    models = {}

    models["LogReg"] = Pipeline([
        ("preprocess", preprocess),
        ("clf", LogisticRegression(max_iter=2000, class_weight="balanced", random_state=RANDOM_STATE))
    ])

    models["DecisionTree"] = Pipeline([
        ("preprocess", preprocess),
        ("clf", DecisionTreeClassifier(class_weight="balanced", random_state=RANDOM_STATE))
    ])

    from lightgbm import LGBMClassifier
    models["LGBM"] = Pipeline([
        ("preprocess", preprocess),
        ("clf", LGBMClassifier(
            n_estimators=400, learning_rate=0.05,
            random_state=RANDOM_STATE,
            is_unbalance=True
        ))
    ])

    from xgboost import XGBClassifier
    models["XGBoost"] = Pipeline([
        ("preprocess", preprocess),
        ("clf", XGBClassifier(
            n_estimators=400, learning_rate=0.05, max_depth=5,
            subsample=0.9, colsample_bytree=0.9,
            eval_metric="logloss",
            random_state=RANDOM_STATE
        ))
    ])

    return models


## 11) Hàm đánh giá chung: Acc/Prec/Rec/F1 + Confusion Matrix + ROC-AUC


In [ ]:
def get_scores_for_roc(model, X_test):
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X_test)[:, 1]
    if hasattr(model, "decision_function"):
        s = model.decision_function(X_test)
        return (s - s.min()) / (s.max() - s.min() + 1e-9)
    return model.predict(X_test)

def plot_confusion(cm, title):
    plt.figure()
    plt.imshow(cm, aspect="auto")
    plt.title(title)
    plt.xlabel("Predicted"); plt.ylabel("True")
    for (i, j), v in np.ndenumerate(cm):
        plt.text(j, i, str(v), ha="center", va="center")
    plt.colorbar()
    plt.tight_layout()
    plt.show()

def plot_roc(fpr, tpr, auc_score, title):
    plt.figure()
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1])
    plt.title(f"{title} (AUC={auc_score:.4f})")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.tight_layout()
    plt.show()

def evaluate_one(name, model, X_train, X_test, y_train, y_test, plot=True):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    scores = get_scores_for_roc(model, X_test)
    fpr, tpr, _ = roc_curve(y_test, scores)
    auc_score = auc(fpr, tpr)

    if plot:
        print(f"\n=== {name} ===")
        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall   : {rec:.4f}")
        print(f"F1-score : {f1:.4f}")
        print(classification_report(y_test, y_pred, zero_division=0))

        cm = confusion_matrix(y_test, y_pred)
        plot_confusion(cm, f"Confusion Matrix - {name}")
        plot_roc(fpr, tpr, auc_score, f"ROC Curve - {name}")

    return {"model": name, "acc": acc, "prec": prec, "rec": rec, "f1": f1, "auc": auc_score}


## 12) Chạy theo nhiều tỉ lệ train/test (0.1, 0.2, 0.3) + bảng so sánh


In [ ]:
y = df["label"]
X = df.drop(columns=["quality", "label"]).copy()

preprocess = make_preprocess(X)
models = build_models(preprocess)

rows = []
for ts in TEST_SIZES:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=ts, stratify=y, random_state=RANDOM_STATE
    )

    print(f"\n\n==================== test_size={ts} ====================")
    for mname, model in models.items():
        r = evaluate_one(f"{mname}_ts{ts}", model, X_train, X_test, y_train, y_test, plot=True)
        r["test_size"] = ts
        r["base_model"] = mname
        rows.append(r)

results_df = pd.DataFrame(rows).sort_values(["test_size","auc"], ascending=[True,False])
display(results_df)

best_each = results_df.sort_values("auc", ascending=False).groupby("test_size").head(1)
print("Best per split (by AUC):")
display(best_each)


## 13) Feature importance (LGBM/XGBoost)


In [ ]:
def get_feature_names(preprocess, X_train):
    num_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

    names = []
    names.extend(num_features)
    if len(cat_features) > 0:
        ohe = preprocess.named_transformers_["cat"].named_steps["onehot"]
        names.extend(ohe.get_feature_names_out(cat_features).tolist())
    return np.array(names, dtype=object)

def plot_importance(pipe, X_train, title, topk=15):
    preprocess = pipe.named_steps["preprocess"]
    clf = pipe.named_steps["clf"]
    feat_names = get_feature_names(preprocess, X_train)

    imp = getattr(clf, "feature_importances_", None)
    if imp is None:
        print("No feature_importances_")
        return

    idx = np.argsort(imp)[::-1][:topk]
    plt.figure(figsize=(10, 6))
    plt.barh(feat_names[idx][::-1], imp[idx][::-1])
    plt.title(title)
    plt.tight_layout()
    plt.show()

ts = 0.2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=ts, stratify=y, random_state=RANDOM_STATE
)

models["LGBM"].fit(X_train, y_train)
plot_importance(models["LGBM"], X_train, "LGBM Feature Importance (Top 15)")

models["XGBoost"].fit(X_train, y_train)
plot_importance(models["XGBoost"], X_train, "XGBoost Feature Importance (Top 15)")


## 14) Outliers: Boxplot AFTER IQR clipping (minh hoạ)


In [ ]:
iqr = IQRClipper(factor=IQR_FACTOR)
X_num = df[feature_cols].to_numpy(dtype=float)
X_clip = iqr.fit_transform(X_num)
df_clip = pd.DataFrame(X_clip, columns=feature_cols)

plt.figure(figsize=(18, 6))
df_clip.boxplot(rot=90)
plt.title("Boxplot AFTER IQR clipping (illustration)")
plt.tight_layout()
plt.show()

print("Ghi chú: IQR clipping giảm ảnh hưởng outliers, KHÔNG loại bỏ mẫu.")


## OPTIONAL 1: So sánh Red vs White (train riêng)


In [ ]:
def holdout_compare(df_sub, test_size=0.2, drop_wine_type=True):
    y_sub = (df_sub["quality"] >= LABEL_THRESHOLD).astype(int)
    X_sub = df_sub.drop(columns=["quality"], errors="ignore").copy()
    if drop_wine_type:
        X_sub = X_sub.drop(columns=["wine_type"], errors="ignore")

    preprocess = make_preprocess(X_sub)
    models_sub = build_models(preprocess)

    X_train, X_test, y_train, y_test = train_test_split(
        X_sub, y_sub, test_size=test_size, stratify=y_sub, random_state=RANDOM_STATE
    )

    out = []
    for mname, model in models_sub.items():
        r = evaluate_one(mname, model, X_train, X_test, y_train, y_test, plot=False)
        r["test_size"] = test_size
        out.append(r)
    return pd.DataFrame(out).sort_values("auc", ascending=False)

red_only = df[df["wine_type"]=="red"].copy()
white_only = df[df["wine_type"]=="white"].copy()

red_res = holdout_compare(red_only, test_size=0.2)
white_res = holdout_compare(white_only, test_size=0.2)

print("RED:")
display(red_res)
print("WHITE:")
display(white_res)

red_res["wine_type"]="red"
white_res["wine_type"]="white"
display(pd.concat([red_res, white_res], ignore_index=True).sort_values(["wine_type","auc"], ascending=[True,False]))


## OPTIONAL 2: Regression dự đoán quality (RMSE/MAE/R2)


In [ ]:
def regression_run(df_all, test_size=0.2):
    y_reg = df_all["quality"].astype(float)
    X_reg = df_all.drop(columns=["quality","label"], errors="ignore").copy()

    preprocess = make_preprocess(X_reg)
    reg = Pipeline([
        ("preprocess", preprocess),
        ("reg", RandomForestRegressor(n_estimators=400, random_state=RANDOM_STATE, n_jobs=-1))
    ])

    X_train, X_test, y_train, y_test = train_test_split(
        X_reg, y_reg, test_size=test_size, random_state=RANDOM_STATE
    )

    reg.fit(X_train, y_train)
    pred = reg.predict(X_test)

    rmse = mean_squared_error(y_test, pred) ** 0.5
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    return {"test_size": test_size, "rmse": rmse, "mae": mae, "r2": r2}

reg_df = pd.DataFrame([regression_run(df, ts) for ts in TEST_SIZES])
display(reg_df)


## OPTIONAL 3: Cross-Validation ROC-AUC (mean±std)


In [ ]:
def cv_auc_models(df_all, n_splits=5):
    y_cv = (df_all["quality"] >= LABEL_THRESHOLD).astype(int)
    X_cv = df_all.drop(columns=["quality","label"], errors="ignore").copy()

    preprocess = make_preprocess(X_cv)
    models_cv = build_models(preprocess)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)

    rows = []
    for mname, pipe in models_cv.items():
        scores = cross_val_score(pipe, X_cv, y_cv, scoring="roc_auc", cv=skf, n_jobs=-1)
        rows.append({
            "model": mname,
            "folds": n_splits,
            "mean_auc": scores.mean(),
            "std_auc": scores.std(),
            "min_auc": scores.min(),
            "max_auc": scores.max()
        })
    return pd.DataFrame(rows).sort_values("mean_auc", ascending=False)

cv_df = cv_auc_models(df, n_splits=CV_FOLDS)
display(cv_df)


## OPTIONAL 4: Hyperparameter tuning (RandomizedSearchCV) cho LGBM


In [ ]:
from lightgbm import LGBMClassifier

y_tune = (df["quality"] >= LABEL_THRESHOLD).astype(int)
X_tune = df.drop(columns=["quality","label"], errors="ignore").copy()

preprocess = make_preprocess(X_tune)

pipe_lgbm = Pipeline([
    ("preprocess", preprocess),
    ("clf", LGBMClassifier(random_state=RANDOM_STATE, is_unbalance=True))
])

param_dist = {
    "clf__n_estimators": [200, 400, 800],
    "clf__learning_rate": [0.03, 0.05, 0.1],
    "clf__max_depth": [-1, 4, 6, 8],
    "clf__num_leaves": [15, 31, 63, 127],
    "clf__subsample": [0.7, 0.85, 1.0],
    "clf__colsample_bytree": [0.7, 0.85, 1.0],
}

skf = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)

rs = RandomizedSearchCV(
    pipe_lgbm,
    param_distributions=param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=skf,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=1
)

rs.fit(X_tune, y_tune)
print("Best AUC:", rs.best_score_)
print("Best params:", rs.best_params_)


# Kết luận & hướng phát triển

- Thường **LGBM/XGBoost** đạt AUC/F1 tốt hơn nhờ boosting học quan hệ phi tuyến.
- Logistic Regression là baseline dễ giải thích nhưng hạn chế.
- IQR clipping giúp giảm tác động outliers mà **không loại bỏ mẫu**.

## Hướng phát triển
- Tuning + CV mean±std sâu hơn, thử SHAP để giải thích.
- Regression nâng cao với LGBMRegressor/XGBRegressor.
- So sánh sâu red vs white.
- Triển khai demo dự đoán.
